In [ ]:
from datetime import datetime as dtime
from tkinter import Tk  # use tkinter to open files
from tkinter.filedialog import askopenfilename, askdirectory

# Define processing functions

In [ ]:
def open_file(title: str = "Select file") -> str:
    """
    Opens a tkinter dialog to select a file. Returns the path of the file.
    :param title: The message to display in the open directory dialog.
    :return: the absolute path of the directory selected.
    """
    root = Tk()
    # dialog should open on top. Only works for Windows?
    root.attributes("-topmost", True)
    root.withdraw()  # keep root window from appearing
    return askopenfilename(title=title)

In [ ]:
def isValidMessage(text_line):
    """
    This function should test if text_line is a valid message, or a continuation or similar.
    A valid line looks like this:
    
    dd/mm/yyyy, hh:mm - Author: Message\n
    
    1. split around "-"
    2. test contents of first half for datetime-like content
    3. test first     
    """
    date_time, _, author_message = map(lambda x: x.strip(), text_line.partition("-"))
    
    # partition returns complete string and two empty strings if partition character not found
    if len(author_message) == 0:  
        return False
    
    author, _, message = map(lambda x: x.strip(), date_time.partition(","))
    date, _, time = map(lambda x: x.strip(), date_time.partition(","))
    
    if (len(message) == 0) or (len(time) == 0):
        return False
    
    datelike = len(date.split("/")) == 3  # check stuff before first comma is a likely date
    timelike = len(time.split(":")) == 2
    
    if datelike and timelike:
        return True
    else:
        return False
    

In [ ]:
def messageToDtimeAuthorMessage(message: str, uid: int = 0):
    """
    Format is assumed to be
    dd/mm/yyyy, hh:mm - Author: Message\n
    
    UID: unique identifier of line. This function just appends it 
    to the tuple and returns this UID (or shows in error messages), 
    does not check if it is valid. 
    
    This function then returns a tuple:
    (uid, datetime: dtime, author: str, message: str)
    
    """
    
    # date_time: "dd/mm/yyyy, hh:mm"
    # author_message: "Author: Message"
    date_time, _, author_message = map(lambda x: x.strip(), message.partition("-"))
    assert len(author_message) > 0, f"{uid}: Author empty. Message: {message}"  # make sure the message could be split in two
    
    date, time = map(lambda x: x.strip(), date_time.split(","))  # apply whitespace strip
    author, _, message = map(lambda x: x.strip(), author_message.partition(":"))
    assert(len(date.split("/")) == 3)
    assert(len(time.split(":")) == 2)
    # print(f"Date is\n{date}\nTime is:\n{time}\nAuthor:\n{author}\nMessage:\n{message}")
    
    ymdhms = list(map(lambda string: int(string), reversed(date.split("/"))))  # [year, month, day, hour, minute, second]
    hm = list(map(lambda string: int(string), time.split(":")))
    ymdhms.append(hm[0])
    ymdhms.append(hm[1])
    ymdhms.append(0)  # second is not recorded in chat. Maybe in future?
    
    message_datetime = dtime(*ymdhms)
    
    return (uid, message_datetime, author, message)

### Test stuff for processing entry in txt file

In [ ]:
s = "15/12/1991, 13:26 - John: Hey Jeff,\n how you doin? A/B - which one do you choose?\n"

In [ ]:
list(map(lambda string: int(string), reversed("15/12/1991".split("/"))))

In [ ]:
a = messageToDtimeAuthorMessage(s, 123)

# Read data from file

In [ ]:
fpath = open_file()
print(f"File chosen: {fpath}")

In [ ]:
isValidMessage("https://maps.app.goo.gl/E4Bvr6DF\n")

In [ ]:
raw_data = []
line_index = 0
with open(fpath, "r") as f:
    for line in f:
        if isValidMessage(line):
            raw_data.append(line)
        else:
            raw_data[-1] += line
        #raw_data.append(messageToDtimeAuthorMessage(line, line_index))
        line_index += 1

In [ ]:
extracted_data = []
line_count = 0
for line in raw_data:
    extracted_data.append(messageToDtimeAuthorMessage(line, line_count))
    line_count += 1

# Work with extracted data

### extracted_data has shape [(uid: int, datetime: datetime, author: str, message: str), ...   ]

## Who wrote more?

In [ ]:
message_counts = dict()
char_counts = dict()

for message_tuple in extracted_data:
    author = message_tuple[2]
    if author in message_counts.keys():
        message_counts[author] += 1
        char_counts[author] += len(message_tuple[3])
    else:
        message_counts[author] = 1
        char_counts[author] = len(message_tuple[3])
        print(f"Added {author}")
        

### Number of messages sent

In [ ]:
num_msg = 0
for author in message_counts.keys():
    num_msg += message_counts[author]
assert len(extracted_data) == num_msg, \
    f"Something wrong, number of messages extracted does not match sum of messages assigned to authors"

In [ ]:
print(f"Analyzed {len(extracted_data)} messages.")
for user in message_counts.keys():
    print(f"{message_counts[user]} messages typed by {user}.")

### Number of characters sent

In [ ]:
print(f"Analyzed {len(extracted_data)} messages.")
for user in message_counts.keys():
    print(f"{char_counts[user]} characters typed by {user}.")

# Get all smileys used

In [ ]:
flattened_string_enc = str.encode("")
flattened_string = ""
for message in extracted_data:
    flattened_string_enc += str.encode(message[3])
    flattened_string += message[3]
assert len(flattened_string) == sum([char_counts[user] for user in char_counts.keys()])

In [ ]:
import re

In [ ]:
#re.findall(rb"\x[0-9A-Fa-f]{2}\x[0-9A-Fa-f]{2}\x[0-9A-Fa-f]{2}\x[0-9A-Fa-f]{2}", flattened_string)

In [ ]:
re.findall(b'\xf0\x9f\x98\x82', flattened_string_enc)

In [ ]:
emojis_dict = {
    "people" : "😀 😃 😄 😁 😆 😅 🤣 😂 🥹 🙂 🙃 😉 😊 😇 🥰 😍 🤩 😘 😗 ☺️ 😚 😙 🥲 😋 😛 😜 🤪 😝 🤑 🤗 🫢 🤭 🤫 🤔 🫡 🤐 🤨 😐️ 😑 😶 😏 😒 🙄 😬 🤥 😌 😔 😪 😮‍💨 🤤 😴 😷 🤒 🤕 🤢 🤮 🤧 🫠 🥵 🥶 😶‍🌫️ 🫥 🥴 😵‍💫 😵 🤯 🤠 🥳 🥸 😎 🤓 🧐 🫤 😕 😟 🙁 ☹️ 😮 😯 😲 😳 🫣 🥺 😦 😧 😨 😰 😥 😢 😭 😱 😖 😣 😞 😓 😩 😫 🥱 😤 😡 😠 🤬 😈 👿 💀 ☠️ 💩 🤡 👹 👺 👻 👽️ 👾 🤖 😺 😸 😹 😻 😼 😽 🙀 😿 😾 🙈 🙉 🙊 👋 🤚 🖐️ ✋ 🖖 👌 🤌 🤏 ✌️ 🤞 🫰 🤟 🤘 🤙 👈️ 👉️ 👆️ 🖕 👇️ ☝️ 🫵 👍️ 👎️ ✊ 👊 🤛 🤜 👏 🙌 👐 🫶 🤲 🫳 🫴 🫱 🫲 🤝 🙏 ✍️ 💅 🤳 💪 🦾 🦿 🦵 🦶 👂️ 🦻 👃 🧠 🫀 🫁 🦷 🦴 👀 👁️ 👅 👄 🫦 💋 👶 🧒 👦 👧 🧑 👨 👩 🧔 🧔‍♀️ 🧔‍♂️ 🧑‍🦰 👨‍🦰 👩‍🦰 🧑‍🦱 👨‍🦱 👩‍🦱 🧑‍🦳 👨‍🦳 👩‍🦳 🧑‍🦲 👨‍🦲 👩‍🦲 👱 👱‍♂️ 👱‍♀️ 🧓 👴 👵 🙍 🙍‍♂️ 🙍‍♀️ 🙎 🙎‍♂️ 🙎‍♀️ 🙅 🙅‍♂️ 🙅‍♀️ 🙆 🙆‍♂️ 🙆‍♀️ 💁 💁‍♂️ 💁‍♀️ 🙋 🙋‍♂️ 🙋‍♀️ 🧏 🧏‍♂️ 🧏‍♀️ 🙇 🙇‍♂️ 🙇‍♀️ 🤦 🤦‍♂️ 🤦‍♀️ 🤷 🤷‍♂️ 🤷‍♀️ 🧑‍⚕️ 👨‍⚕️ 👩‍⚕️ 🧑‍🎓 👨‍🎓 👩‍🎓 🧑‍🏫 👨‍🏫 👩‍🏫 🧑‍⚖️ 👨‍⚖️ 👩‍⚖️ 🧑‍🌾 👨‍🌾 👩‍🌾 🧑‍🍳 👨‍🍳 👩‍🍳 🧑‍🔧 👨‍🔧 👩‍🔧 🧑‍🏭 👨‍🏭 👩‍🏭 🧑‍💼 👨‍💼 👩‍💼 🧑‍🔬 👨‍🔬 👩‍🔬 🧑‍💻 👨‍💻 👩‍💻 🧑‍🎤 👨‍🎤 👩‍🎤 🧑‍🎨 👨‍🎨 👩‍🎨 🧑‍✈️ 👨‍✈️ 👩‍✈️ 🧑‍🚀 👨‍🚀 👩‍🚀 🧑‍🚒 👨‍🚒 👩‍🚒 👮 👮‍♂️ 👮‍♀️ 🕵️ 🕵️‍♂️ 🕵️‍♀️ 💂 💂‍♂️ 💂‍♀️ 🥷 👷 👷‍♂️ 👷‍♀️ 🫅 🤴 👸 👳 👳‍♂️ 👳‍♀️ 👲 🧕 🤵 🤵‍♂️ 🤵‍♀️ 👰 👰‍♂️ 👰‍♀️ 🫄 🫃 🤰 🤱 👩‍🍼 👨‍🍼 🧑‍🍼 👼 🎅 🤶 🧑‍🎄 🦸 🦸‍♂️ 🦸‍♀️ 🦹 🦹‍♂️ 🦹‍♀️ 🧙 🧙‍♂️ 🧙‍♀️ 🧚 🧚‍♂️ 🧚‍♀️ 🧛 🧛‍♂️ 🧛‍♀️ 🧜 🧜‍♂️ 🧜‍♀️ 🧝 🧝‍♂️ 🧝‍♀️ 🧞 🧞‍♂️ 🧞‍♀️ 🧟 🧟‍♂️ 🧟‍♀️ 🧌 💆 💆‍♂️ 💆‍♀️ 💇 💇‍♂️ 💇‍♀️ 🚶 🚶‍♂️ 🚶‍♀️ 🧍 🧍‍♂️ 🧍‍♀️ 🧎 🧎‍♂️ 🧎‍♀️ 🧑‍🦯 👨‍🦯 👩‍🦯 🧑‍🦼 👨‍🦼 👩‍🦼 🧑‍🦽 👨‍🦽 👩‍🦽 🏃 🏃‍♂️ 🏃‍♀️ 💃 🕺 🕴️ 👯 👯‍♂️ 👯‍♀️ 🧖 🧖‍♂️ 🧖‍♀️ 🧗 🧗‍♂️ 🧗‍♀️ 🤺 🏇 ⛷️ 🏂️ 🏌️ 🏌️‍♂️ 🏌️‍♀️ 🏄️ 🏄‍♂️ 🏄‍♀️ 🚣 🚣‍♂️ 🚣‍♀️ 🏊️ 🏊‍♂️ 🏊‍♀️ ⛹️ ⛹️‍♂️ ⛹️‍♀️ 🏋️ 🏋️‍♂️ 🏋️‍♀️ 🚴 🚴‍♂️ 🚴‍♀️ 🚵 🚵‍♂️ 🚵‍♀️ 🤸 🤸‍♂️ 🤸‍♀️ 🤼 🤼‍♂️ 🤼‍♀️ 🤽 🤽‍♂️ 🤽‍♀️ 🤾 🤾‍♂️ 🤾‍♀️ 🤹 🤹‍♂️ 🤹‍♀️ 🧘 🧘‍♂️ 🧘‍♀️ 🛀 🛌 🧑‍🤝‍🧑 👭 👫 👬 💏 👩‍❤️‍💋‍👨 👨‍❤️‍💋‍👨 👩‍❤️‍💋‍👩 💑 👩‍❤️‍👨 👨‍❤️‍👨 👩‍❤️‍👩 👪️ 👨‍👩‍👦 👨‍👩‍👧 👨‍👩‍👧‍👦 👨‍👩‍👦‍👦 👨‍👩‍👧‍👧 👨‍👨‍👦 👨‍👨‍👧 👨‍👨‍👧‍👦 👨‍👨‍👦‍👦 👨‍👨‍👧‍👧 👩‍👩‍👦 👩‍👩‍👧 👩‍👩‍👧‍👦 👩‍👩‍👦‍👦 👩‍👩‍👧‍👧 👨‍👦 👨‍👦‍👦 👨‍👧 👨‍👧‍👦 👨‍👧‍👧 👩‍👦 👩‍👦‍👦 👩‍👧 👩‍👧‍👦 👩‍👧‍👧 🗣️ 👤 👥 🫂 👣",
    "animals nature" :"🐵 🐒 🦍 🦧 🐶 🐕️ 🦮 🐕‍🦺 🐩 🐺 🦊 🦝 🐱 🐈️ 🐈‍⬛ 🦁 🐯 🐅 🐆 🐴 🐎 🦄 🦓 🦌 🦬 🐮 🐂 🐃 🐄 🐷 🐖 🐗 🐽 🐏 🐑 🐐 🐪 🐫 🦙 🦒 🐘 🦣 🦏 🦛 🐭 🐁 🐀 🐹 🐰 🐇 🐿️ 🦫 🦔 🦇 🐻 🐻‍❄️ 🐨 🐼 🦥 🦦 🦨 🦘 🦡 🐾 🦃 🐔 🐓 🐣 🐤 🐥 🐦️ 🐧 🕊️ 🦅 🦆 🦢 🦉 🦤 🪶 🪹 🪺 🦩 🦚 🦜 🐸 🐊 🐢 🦎 🐍 🐲 🐉 🦕 🦖 🐳 🐋 🐬 🦭 🐟️ 🐠 🐡 🦈 🐙 🦑 🦀 🦞 🦐 🪸 🦪 🐚 🐌 🦋 🐛 🐜 🐝 🪲 🐞 🦗 🪳 🕷️ 🕸️ 🦂 🦟 🪰 🪱 🦠 💐 🌸 💮 🏵️ 🌹 🥀 🌺 🪷 🌻 🌼 🌷 🌱 🪴 🌲 🌳 🌴 🌵 🎋 🎍 🌾 🌿 ☘️ 🍀 🍁 🍂 🍃 🌍️ 🌎️ 🌏️ 🌑 🌒 🌓 🌔 🌕️ 🌖 🌗 🌘 🌙 🌚 🌛 🌜️ ☀️ 🌝 🌞 🪐 💫 ⭐️ 🌟 ✨ 🌠 🌌 ☁️ ⛅️ ⛈️ 🌤️ 🌥️ 🌦️ 🌧️ 🌨️ 🌩️ 🌪️ 🌫️ 🌬️ 🌀 🌈 🌂 ☂️ ☔️ ⛱️ ⚡️ ❄️ ☃️ ⛄️ ☄️ 🔥 💧 🌊 💥 💦 💨",
    "food drink" : "🍇 🍈 🍉 🍊 🍋 🍌 🍍 🥭 🍎 🍏 🍐 🍑 🍒 🍓 🫐 🥝 🍅 🫒 🥥 🥑 🍆 🥔 🥕 🌽 🌶️ 🫑 🥒 🥬 🥦 🧄 🧅 🍄 🫘 🥜 🌰 🍞 🥐 🥖 🫓 🥨 🥯 🥞 🧇 🧀 🍖 🍗 🥩 🥓 🍔 🍟 🍕 🌭 🥪 🌮 🌯 🫔 🥙 🧆 🥚 🍳 🥘 🍲 🫕 🥣 🥗 🍿 🧈 🧂 🥫 🍱 🍘 🍙 🍚 🍛 🍜 🍝 🍠 🍢 🍣 🍤 🍥 🥮 🍡 🥟 🥠 🥡 🍦 🍧 🍨 🍩 🍪 🎂 🍰 🧁 🥧 🍫 🍬 🍭 🍮 🍯 🍼 🥛 🫗 ☕️ 🫖 🍵 🍶 🍾 🍷 🍸️ 🍹 🍺 🍻 🥂 🥃 🥤 🧋 🧃 🧉 🧊 🥢 🍽️ 🍴 🥄 🔪",
    "sports activities" : "⚽️ ⚾️ 🥎 🏀 🏐 🏈 🏉 🎾 🥏 🎳 🏏 🏑 🏒 🥍 🏓 🏸 🥊 🥋 🥅 ⛳️ ⛸️ 🎣 🤿 🎽 🎿 🛷 🥌 🎯 🪀 🪁 🎱 🎖️ 🏆️ 🏅 🥇 🥈 🥉",
    "vehicles travel" : "🏔️ ⛰️ 🌋 🗻 🏕️ 🏖️ 🏜️ 🏝️ 🏟️ 🏛️ 🏗️ 🧱 🪨 🪵 🛖 🏘️ 🏚️ 🏠️ 🏡 🏢 🏣 🏤 🏥 🏦 🏨 🏩 🏪 🏫 🏬 🏭️ 🏯 🏰 💒 🗼 🗽 ⛪️ 🕌 🛕 🕍 ⛩️ 🕋 ⛲️ ⛺️ 🌁 🌃 🏙️ 🌄 🌅 🌆 🌇 🌉 🗾 🏞️ 🎠 🎡 🎢 💈 🎪 🚂 🚃 🚄 🚅 🚆 🚇️ 🚈 🚉 🚊 🚝 🚞 🚋 🚌 🚍️ 🚎 🚐 🚑️ 🚒 🚓 🚔️ 🚕 🚖 🚗 🚘️ 🚙 🛻 🚚 🚛 🚜 🏎️ 🏍️ 🛵 🦽 🦼 🛺 🚲️ 🛴 🛹 🛼 🚏 🛣️ 🛤️ 🛢️ ⛽️ 🚨 🚥 🚦 🛑 🚧 ⚓️ ⛵️ 🛶 🚤 🛳️ ⛴️ 🛥️ 🚢 ✈️ 🛩️ 🛫 🛬 🪂 💺 🚁 🚟 🚠 🚡 🛰️ 🚀 🛸 🎆 🎇 🎑 🗿",
    "objects" : "🛎️ 🧳 ⌛️ ⏳️ ⌚️ ⏰ ⏱️ ⏲️ 🕰️ 🌡️ 🗺️ 🧭 🎃 🎄 🧨 🎈 🎉 🎊 🎎 🎏 🎐 🎀 🎁 🎗️ 🎟️ 🎫 🔮 🪄 🧿 🎮️ 🕹️ 🎰 🎲 ♟️ 🧩 🧸 🪅 🪆 🖼️ 🎨 🧵 🪡 🧶 🪢 👓️ 🕶️ 🥽 🥼 🦺 👔 👕 👖 🧣 🧤 🧥 🧦 👗 👘 🥻 🩱 🩲 🩳 👙 👚 👛 👜 👝 🛍️ 🎒 🩴 👞 👟 🥾 🥿 👠 👡 🩰 👢 👑 👒 🎩 🎓️ 🧢 🪖 ⛑️ 📿 💄 💍 💎 📢 📣 📯 🎙️ 🎚️ 🎛️ 🎤 🎧️ 📻️ 🎷 🪗 🎸 🎹 🎺 🎻 🪕 🥁 🪘 🪩 📱 📲 ☎️ 📞 📟️ 📠 🔋 🪫 🔌 💻️ 🖥️ 🖨️ ⌨️ 🖱️ 🖲️ 💽 💾 💿️ 📀 🧮 🎥 🎞️ 📽️ 🎬️ 📺️ 📷️ 📸 📹️ 📼 🔍️ 🔎 🕯️ 💡 🔦 🏮 🪔 📔 📕 📖 📗 📘 📙 📚️ 📓 📒 📃 📜 📄 📰 🗞️ 📑 🔖 🏷️ 💰️ 🪙 💴 💵 💶 💷 💸 💳️ 🪪 🧾 ✉️ 💌 📧 🧧 📨 📩 📤️ 📥️ 📦️ 📫️ 📪️ 📬️ 📭️ 📮 🗳️ ✏️ ✒️ 🖋️ 🖊️ 🖌️ 🖍️ 📝 💼 📁 📂 🗂️ 📅 📆 🗒️ 🗓️ 📇 📈 📉 📊 📋️ 📌 📍 📎 🖇️ 📏 📐 ✂️ 🗃️ 🗄️ 🗑️ 🔒️ 🔓️ 🔏 🔐 🔑 🗝️ 🔨 🪓 ⛏️ ⚒️ 🛠️ 🗡️ ⚔️ 💣️ 🔫 🪃 🏹 🛡️ 🪚 🔧 🪛 🔩 ⚙️ 🗜️ ⚖️ 🦯 🔗 ⛓️ 🪝 🧰 🧲 🪜 🛝 🛞 🫙 ⚗️ 🧪 🧫 🧬 🔬 🔭 📡 🩻 💉 🩸 💊 🩹 🩺 🩼 🚪 🛗 🪞 🪟 🛏️ 🛋️ 🪑 🪤 🚽 🪠 🚿 🛁 🧼 🫧 🪒 🧴 🧷 🧹 🧺 🧻 🪣 🪥 🧽 🧯 🛟 🛒 🚬 ⚰️ 🪦 ⚱️ 🏺 🪧 🕳️",
    "symbols" : "💘 💝 💖 💗 💓 💞 💕 💟 ❣️ 💔 ❤️ 🧡 💛 💚 💙 💜 🤎 🖤 🤍 ❤️‍🔥 ❤️‍🩹 💯 ♨️ 💢 💬 👁️‍🗨️ 🗨️ 🗯️ 💭 💤 🌐 ♠️ ♥️ ♦️ ♣️ 🃏 🀄️ 🎴 🎭️ 🔇 🔈️ 🔉 🔊 🔔 🔕 🎼 🎵 🎶 💹 🏧 🚮 🚰 ♿️ 🚹️ 🚺️ 🚻 🚼️ 🚾 🛂 🛃 🛄 🛅 ⚠️ 🚸 ⛔️ 🚫 🚳 🚭️ 🚯 🚱 🚷 📵 🔞 ☢️ ☣️ ⬆️ ↗️ ➡️ ↘️ ⬇️ ↙️ ⬅️ ↖️ ↕️ ↔️ ↩️ ↪️ ⤴️ ⤵️ 🔃 🔄 🔙 🔚 🔛 🔜 🔝 🛐 ⚛️ 🕉️ ✡️ ☸️ ☯️ ✝️ ☦️ ☪️ ☮️ 🕎 🔯 🪬 ♈️ ♉️ ♊️ ♋️ ♌️ ♍️ ♎️ ♏️ ♐️ ♑️ ♒️ ♓️ ⛎ 🔀 🔁 🔂 ▶️ ⏩️ ⏭️ ⏯️ ◀️ ⏪️ ⏮️ 🔼 ⏫ 🔽 ⏬ ⏸️ ⏹️ ⏺️ ⏏️ 🎦 🔅 🔆 📶 📳 📴 ♀️ ♂️ ⚧ ✖️ ➕ ➖ ➗ 🟰 ♾️ ‼️ ⁉️ ❓️ ❔ ❕ ❗️ 〰️ 💱 💲 ⚕️ ♻️ ⚜️ 🔱 📛 🔰 ⭕️ ✅ ☑️ ✔️ ❌ ❎ ➰ ➿ 〽️ ✳️ ✴️ ❇️ ©️ ®️ ™️ #️⃣ *️⃣ 0️⃣ 1️⃣ 2️⃣ 3️⃣ 4️⃣ 5️⃣ 6️⃣ 7️⃣ 8️⃣ 9️⃣ 🔟 🔠 🔡 🔢 🔣 🔤 🅰️ 🆎 🅱️ 🆑 🆒 🆓 ℹ️ 🆔 Ⓜ️ 🆕 🆖 🅾️ 🆗 🅿️ 🆘 🆙 🆚 🈁 🈂️ 🈷️ 🈶 🈯️ 🉐 🈹 🈚️ 🈲 🉑 🈸 🈴 🈳 ㊗️ ㊙️ 🈺 🈵 🔴 🟠 🟡 🟢 🔵 🟣 🟤 ⚫️ ⚪️ 🟥 🟧 🟨 🟩 🟦 🟪 🟫 ⬛️ ⬜️ ◼️ ◻️ ◾️ ◽️ ▪️ ▫️ 🔶 🔷 🔸 🔹 🔺 🔻 💠 🔘 🔳 🔲 🕛️ 🕧️ 🕐️ 🕜️ 🕑️ 🕝️ 🕒️ 🕞️ 🕓️ 🕟️ 🕔️ 🕠️ 🕕️ 🕡️ 🕖️ 🕢️ 🕗️ 🕣️ 🕘️ 🕤️ 🕙️ 🕥️ 🕚️ 🕦️ *️ #️ 0️ 1️ 2️ 3️ 4️ 5️ 6️ 7️ 8️ 9️",
    "special" : "🏻 🏼 🏽 🏾 🏿 ♀️ ♂️ ⚕ 🦲 🦱 🦰 🦳",
    "regional indicators" : "🇦 🇧 🇨 🇩 🇪 🇫 🇬 🇭 🇮 🇯 🇰 🇱 🇲 🇳 🇴 🇵 🇶 🇷 🇸 🇹 🇺 🇻 🇼 🇽 🇾 🇿",
    "flags" : "🏁 🚩 🎌 🏴 🏳️ 🏳️‍🌈 🏳️‍⚧️ 🏴‍☠️ 🇦🇨 🇦🇩 🇦🇪 🇦🇫 🇦🇬 🇦🇮 🇦🇱 🇦🇲 🇦🇴 🇦🇶 🇦🇷 🇦🇸 🇦🇹 🇦🇺 🇦🇼 🇦🇽 🇦🇿 🇧🇦 🇧🇧 🇧🇩 🇧🇪 🇧🇫 🇧🇬 🇧🇭 🇧🇮 🇧🇯 🇧🇱 🇧🇲 🇧🇳 🇧🇴 🇧🇶 🇧🇷 🇧🇸 🇧🇹 🇧🇻 🇧🇼 🇧🇾 🇧🇿 🇨🇦 🇨🇨 🇨🇩 🇨🇫 🇨🇬 🇨🇭 🇨🇮 🇨🇰 🇨🇱 🇨🇲 🇨🇳 🇨🇴 🇨🇵 🇨🇷 🇨🇺 🇨🇻 🇨🇼 🇨🇽 🇨🇾 🇨🇿 🇩🇪 🇩🇬 🇩🇯 🇩🇰 🇩🇲 🇩🇴 🇩🇿 🇪🇦 🇪🇨 🇪🇪 🇪🇬 🇪🇭 🇪🇷 🇪🇸 🇪🇹 🇪🇺 🇫🇮 🇫🇯 🇫🇰 🇫🇲 🇫🇴 🇫🇷 🇬🇦 🇬🇧 🇬🇩 🇬🇪 🇬🇫 🇬🇬 🇬🇭 🇬🇮 🇬🇱 🇬🇲 🇬🇳 🇬🇵 🇬🇶 🇬🇷 🇬🇸 🇬🇹 🇬🇺 🇬🇼 🇬🇾 🇭🇰 🇭🇲 🇭🇳 🇭🇷 🇭🇹 🇭🇺 🇮🇨 🇮🇩 🇮🇪 🇮🇱 🇮🇲 🇮🇳 🇮🇴 🇮🇶 🇮🇷 🇮🇸 🇮🇹 🇯🇪 🇯🇲 🇯🇴 🇯🇵 🇰🇪 🇰🇬 🇰🇭 🇰🇮 🇰🇲 🇰🇳 🇰🇵 🇰🇷 🇰🇼 🇰🇾 🇰🇿 🇱🇦 🇱🇧 🇱🇨 🇱🇮 🇱🇰 🇱🇷 🇱🇸 🇱🇹 🇱🇺 🇱🇻 🇱🇾 🇲🇦 🇲🇨 🇲🇩 🇲🇪 🇲🇫 🇲🇬 🇲🇭 🇲🇰 🇲🇱 🇲🇲 🇲🇳 🇲🇴 🇲🇵 🇲🇶 🇲🇷 🇲🇸 🇲🇹 🇲🇺 🇲🇻 🇲🇼 🇲🇽 🇲🇾 🇲🇿 🇳🇦 🇳🇨 🇳🇪 🇳🇫 🇳🇬 🇳🇮 🇳🇱 🇳🇴 🇳🇵 🇳🇷 🇳🇺 🇳🇿 🇴🇲 🇵🇦 🇵🇪 🇵🇫 🇵🇬 🇵🇭 🇵🇰 🇵🇱 🇵🇲 🇵🇳 🇵🇷 🇵🇸 🇵🇹 🇵🇼 🇵🇾 🇶🇦 🇷🇪 🇷🇴 🇷🇸 🇷🇺 🇷🇼 🇸🇦 🇸🇧 🇸🇨 🇸🇩 🇸🇪 🇸🇬 🇸🇭 🇸🇮 🇸🇯 🇸🇰 🇸🇱 🇸🇲 🇸🇳 🇸🇴 🇸🇷 🇸🇸 🇸🇹 🇸🇻 🇸🇽 🇸🇾 🇸🇿 🇹🇦 🇹🇨 🇹🇩 🇹🇫 🇹🇬 🇹🇭 🇹🇯 🇹🇰 🇹🇱 🇹🇲 🇹🇳 🇹🇴 🇹🇷 🇹🇹 🇹🇻 🇹🇼 🇹🇿 🇺🇦 🇺🇬 🇺🇳 🇺🇸 🇺🇾 🇺🇿 🇻🇦 🇻🇨 🇻🇪 🇻🇬 🇻🇮 🇻🇳 🇻🇺 🇼🇫 🇼🇸 🇽🇰 🇾🇪 🇾🇹 🇿🇦 🇿🇲 🇿🇼 🏴󠁧󠁢󠁥󠁮󠁧󠁿 🏴󠁧󠁢󠁳󠁣󠁴󠁿 🏴󠁧󠁢󠁷󠁬󠁳󠁿",
    "pale skin" : "👋🏻 🤚🏻 🖐🏻 ✋🏻 🖖🏻 👌🏻 🤌🏻 🤏🏻 ✌🏻 🤞🏻 🤟🏻 🤘🏻 🤙🏻 👈🏻 👉🏻 👆🏻 🖕🏻 👇🏻 ☝🏻 👍🏻 👎🏻 ✊🏻 👊🏻 🤛🏻 🤜🏻 👏🏻 🙌🏻 👐🏻 🤲🏻 🙏🏻 ✍🏻 💅🏻 🤳🏻 💪🏻 🦵🏻 🦶🏻 👂🏻 🦻🏻 👃🏻 👶🏻 🧒🏻 👦🏻 👧🏻 🧑🏻 👱🏻 👨🏻 🧔🏻 🧔🏻‍♀️ 🧔🏻‍♂️ 👨🏻‍🦰 👨🏻‍🦱 👨🏻‍🦳 👨🏻‍🦲 👩🏻 👩🏻‍🦰 🧑🏻‍🦰 👩🏻‍🦱 🧑🏻‍🦱 👩🏻‍🦳 🧑🏻‍🦳 👩🏻‍🦲 🧑🏻‍🦲 👱🏻‍♀️ 👱🏻‍♂️ 🧓🏻 👴🏻 👵🏻 🙍🏻 🙍🏻‍♂️ 🙍🏻‍♀️ 🙎🏻 🙎🏻‍♂️ 🙎🏻‍♀️ 🙅🏻 🙅🏻‍♂️ 🙅🏻‍♀️ 🙆🏻 🙆🏻‍♂️ 🙆🏻‍♀️ 💁🏻 💁🏻‍♂️ 💁🏻‍♀️ 🙋🏻 🙋🏻‍♂️ 🙋🏻‍♀️ 🧏🏻 🧏🏻‍♂️ 🧏🏻‍♀️ 🙇🏻 🙇🏻‍♂️ 🙇🏻‍♀️ 🤦🏻 🤦🏻‍♂️ 🤦🏻‍♀️ 🤷🏻 🤷🏻‍♂️ 🤷🏻‍♀️ 🧑🏻‍⚕️ 👨🏻‍⚕️ 👩🏻‍⚕️ 🧑🏻‍🎓 👨🏻‍🎓 👩🏻‍🎓 🧑🏻‍🏫 👨🏻‍🏫 👩🏻‍🏫 🧑🏻‍⚖️ 👨🏻‍⚖️ 👩🏻‍⚖️ 🧑🏻‍🌾 👨🏻‍🌾 👩🏻‍🌾 🧑🏻‍🍳 👨🏻‍🍳 👩🏻‍🍳 🧑🏻‍🔧 👨🏻‍🔧 👩🏻‍🔧 🧑🏻‍🏭 👨🏻‍🏭 👩🏻‍🏭 🧑🏻‍💼 👨🏻‍💼 👩🏻‍💼 🧑🏻‍🔬 👨🏻‍🔬 👩🏻‍🔬 🧑🏻‍💻 👨🏻‍💻 👩🏻‍💻 🧑🏻‍🎤 👨🏻‍🎤 👩🏻‍🎤 🧑🏻‍🎨 👨🏻‍🎨 👩🏻‍🎨 🧑🏻‍✈️ 👨🏻‍✈️ 👩🏻‍✈️ 🧑🏻‍🚀 👨🏻‍🚀 👩🏻‍🚀 🧑🏻‍🚒 👨🏻‍🚒 👩🏻‍🚒 👮🏻 👮🏻‍♂️ 👮🏻‍♀️ 🕵🏻 🕵🏻‍♂️ 🕵🏻‍♀️ 💂🏻 💂🏻‍♂️ 💂🏻‍♀️ 🥷🏻 👷🏻 👷🏻‍♂️ 👷🏻‍♀️ 🤴🏻 👸🏻 👳🏻 👳🏻‍♂️ 👳🏻‍♀️ 👲🏻 🧕🏻 🤵🏻 🤵🏻‍♂️ 🤵🏻‍♀️ 👰🏻 👰🏻‍♂️ 👰🏻‍♀️ 🤰🏻 🤱🏻 👩🏻‍🍼 👨🏻‍🍼 🧑🏻‍🍼 👼🏻 🎅🏻 🤶🏻 🧑🏻‍🎄 🦸🏻 🦸🏻‍♂️ 🦸🏻‍♀️ 🦹🏻 🦹🏻‍♂️ 🦹🏻‍♀️ 🧙🏻 🧙🏻‍♂️ 🧙🏻‍♀️ 🧚🏻 🧚🏻‍♂️ 🧚🏻‍♀️ 🧛🏻 🧛🏻‍♂️ 🧛🏻‍♀️ 🧜🏻 🧜🏻‍♂️ 🧜🏻‍♀️ 🧝🏻 🧝🏻‍♂️ 🧝🏻‍♀️ 💆🏻 💆🏻‍♂️ 💆🏻‍♀️ 💇🏻 💇🏻‍♂️ 💇🏻‍♀️ 🚶🏻 🚶🏻‍♂️ 🚶🏻‍♀️ 🧍🏻 🧍🏻‍♂️ 🧍🏻‍♀️ 🧎🏻 🧎🏻‍♂️ 🧎🏻‍♀️ 🧑🏻‍🦯 👨🏻‍🦯 👩🏻‍🦯 🧑🏻‍🦼 👨🏻‍🦼 👩🏻‍🦼 🧑🏻‍🦽 👨🏻‍🦽 👩🏻‍🦽 🏃🏻 🏃🏻‍♂️ 🏃🏻‍♀️ 💃🏻 🕺🏻 🕴🏻 🧖🏻 🧖🏻‍♂️ 🧖🏻‍♀️ 🧗🏻 🧗🏻‍♂️ 🧗🏻‍♀️ 🏇🏻 🏂🏻 🏌🏻 🏌🏻‍♂️ 🏌🏻‍♀️ 🏄🏻 🏄🏻‍♂️ 🏄🏻‍♀️ 🚣🏻 🚣🏻‍♂️ 🚣🏻‍♀️ 🏊🏻 🏊🏻‍♂️ 🏊🏻‍♀️ ⛹🏻 ⛹🏻‍♂️ ⛹🏻‍♀️ 🏋🏻 🏋🏻‍♂️ 🏋🏻‍♀️ 🚴🏻 🚴🏻‍♂️ 🚴🏻‍♀️ 🚵🏻 🚵🏻‍♂️ 🚵🏻‍♀️ 🤸🏻 🤸🏻‍♂️ 🤸🏻‍♀️ 🤽🏻 🤽🏻‍♂️ 🤽🏻‍♀️ 🤾🏻 🤾🏻‍♂️ 🤾🏻‍♀️ 🤹🏻 🤹🏻‍♂️ 🤹🏻‍♀️ 🧘🏻 🧘🏻‍♂️ 🧘🏻‍♀️ 🛀🏻 🛌🏻 💑🏻 💏🏻 👫🏻 👭🏻 👬🏻 🫱🏻 🫲🏻 🫳🏻 🫴🏻 🫰🏻 🫵🏻 🫵🏻 🫶🏻 🤝🏻 🫅🏻 🫃🏻 🫄🏻",
    "cream skin" : "👋🏼 🤚🏼 🖐🏼 ✋🏼 🖖🏼 👌🏼 🤌🏼 🤏🏼 ✌🏼 🤞🏼 🤟🏼 🤘🏼 🤙🏼 👈🏼 👉🏼 👆🏼 🖕🏼 👇🏼 ☝🏼 👍🏼 👎🏼 ✊🏼 👊🏼 🤛🏼 🤜🏼 👏🏼 🙌🏼 👐🏼 🤲🏼 🙏🏼 ✍🏼 💅🏼 🤳🏼 💪🏼 🦵🏼 🦶🏼 👂🏼 🦻🏼 👃🏼 👶🏼 🧒🏼 👦🏼 👧🏼 🧑🏼 👱🏼 👨🏼 🧔🏼 🧔🏼‍♀️ 🧔🏼‍♂️ 👨🏼‍🦰 👨🏼‍🦱 👨🏼‍🦳 👨🏼‍🦲 👩🏼 👩🏼‍🦰 🧑🏼‍🦰 👩🏼‍🦱 🧑🏼‍🦱 👩🏼‍🦳 🧑🏼‍🦳 👩🏼‍🦲 🧑🏼‍🦲 👱🏼‍♀️ 👱🏼‍♂️ 🧓🏼 👴🏼 👵🏼 🙍🏼 🙍🏼‍♂️ 🙍🏼‍♀️ 🙎🏼 🙎🏼‍♂️ 🙎🏼‍♀️ 🙅🏼 🙅🏼‍♂️ 🙅🏼‍♀️ 🙆🏼 🙆🏼‍♂️ 🙆🏼‍♀️ 💁🏼 💁🏼‍♂️ 💁🏼‍♀️ 🙋🏼 🙋🏼‍♂️ 🙋🏼‍♀️ 🧏🏼 🧏🏼‍♂️ 🧏🏼‍♀️ 🙇🏼 🙇🏼‍♂️ 🙇🏼‍♀️ 🤦🏼 🤦🏼‍♂️ 🤦🏼‍♀️ 🤷🏼 🤷🏼‍♂️ 🤷🏼‍♀️ 🧑🏼‍⚕️ 👨🏼‍⚕️ 👩🏼‍⚕️ 🧑🏼‍🎓 👨🏼‍🎓 👩🏼‍🎓 🧑🏼‍🏫 👨🏼‍🏫 👩🏼‍🏫 🧑🏼‍⚖️ 👨🏼‍⚖️ 👩🏼‍⚖️ 🧑🏼‍🌾 👨🏼‍🌾 👩🏼‍🌾 🧑🏼‍🍳 👨🏼‍🍳 👩🏼‍🍳 🧑🏼‍🔧 👨🏼‍🔧 👩🏼‍🔧 🧑🏼‍🏭 👨🏼‍🏭 👩🏼‍🏭 🧑🏼‍💼 👨🏼‍💼 👩🏼‍💼 🧑🏼‍🔬 👨🏼‍🔬 👩🏼‍🔬 🧑🏼‍💻 👨🏼‍💻 👩🏼‍💻 🧑🏼‍🎤 👨🏼‍🎤 👩🏼‍🎤 🧑🏼‍🎨 👨🏼‍🎨 👩🏼‍🎨 🧑🏼‍✈️ 👨🏼‍✈️ 👩🏼‍✈️ 🧑🏼‍🚀 👨🏼‍🚀 👩🏼‍🚀 🧑🏼‍🚒 👨🏼‍🚒 👩🏼‍🚒 👮🏼 👮🏼‍♂️ 👮🏼‍♀️ 🕵🏼 🕵🏼‍♂️ 🕵🏼‍♀️ 💂🏼 💂🏼‍♂️ 💂🏼‍♀️ 🥷🏼 👷🏼 👷🏼‍♂️ 👷🏼‍♀️ 🤴🏼 👸🏼 👳🏼 👳🏼‍♂️ 👳🏼‍♀️ 👲🏼 🧕🏼 🤵🏼 🤵🏼‍♂️ 🤵🏼‍♀️ 👰🏼 👰🏼‍♂️ 👰🏼‍♀️ 🤰🏼 🤱🏼 👩🏼‍🍼 👨🏼‍🍼 🧑🏼‍🍼 👼🏼 🎅🏼 🤶🏼 🧑🏼‍🎄 🦸🏼 🦸🏼‍♂️ 🦸🏼‍♀️ 🦹🏼 🦹🏼‍♂️ 🦹🏼‍♀️ 🧙🏼 🧙🏼‍♂️ 🧙🏼‍♀️ 🧚🏼 🧚🏼‍♂️ 🧚🏼‍♀️ 🧛🏼 🧛🏼‍♂️ 🧛🏼‍♀️ 🧜🏼 🧜🏼‍♂️ 🧜🏼‍♀️ 🧝🏼 🧝🏼‍♂️ 🧝🏼‍♀️ 💆🏼 💆🏼‍♂️ 💆🏼‍♀️ 💇🏼 💇🏼‍♂️ 💇🏼‍♀️ 🚶🏼 🚶🏼‍♂️ 🚶🏼‍♀️ 🧍🏼 🧍🏼‍♂️ 🧍🏼‍♀️ 🧎🏼 🧎🏼‍♂️ 🧎🏼‍♀️ 🧑🏼‍🦯 👨🏼‍🦯 👩🏼‍🦯 🧑🏼‍🦼 👨🏼‍🦼 👩🏼‍🦼 🧑🏼‍🦽 👨🏼‍🦽 👩🏼‍🦽 🏃🏼 🏃🏼‍♂️ 🏃🏼‍♀️ 💃🏼 🕺🏼 🕴🏼 🧖🏼 🧖🏼‍♂️ 🧖🏼‍♀️ 🧗🏼 🧗🏼‍♂️ 🧗🏼‍♀️ 🏇🏼 🏂🏼 🏌🏼 🏌🏼‍♂️ 🏌🏼‍♀️ 🏄🏼 🏄🏼‍♂️ 🏄🏼‍♀️ 🚣🏼 🚣🏼‍♂️ 🚣🏼‍♀️ 🏊🏼 🏊🏼‍♂️ 🏊🏼‍♀️ ⛹🏼 ⛹🏼‍♂️ ⛹🏼‍♀️ 🏋🏼 🏋🏼‍♂️ 🏋🏼‍♀️ 🚴🏼 🚴🏼‍♂️ 🚴🏼‍♀️ 🚵🏼 🚵🏼‍♂️ 🚵🏼‍♀️ 🤸🏼 🤸🏼‍♂️ 🤸🏼‍♀️ 🤽🏼 🤽🏼‍♂️ 🤽🏼‍♀️ 🤾🏼 🤾🏼‍♂️ 🤾🏼‍♀️ 🤹🏼 🤹🏼‍♂️ 🤹🏼‍♀️ 🧘🏼 🧘🏼‍♂️ 🧘🏼‍♀️ 🛀🏼 🛌🏼 💑🏼 💏🏼 👫🏼 👭🏼 👬🏼 🫱🏼 🫲🏼 🫳🏼 🫴🏼 🫰🏼 🫵🏼 🫵🏼 🫶🏼 🤝🏼 🫅🏼 🫃🏼 🫄🏼",
    "brown skin" : "👋🏽 🤚🏽 🖐🏽 ✋🏽 🖖🏽 👌🏽 🤌🏽 🤏🏽 ✌🏽 🤞🏽 🤟🏽 🤘🏽 🤙🏽 👈🏽 👉🏽 👆🏽 🖕🏽 👇🏽 ☝🏽 👍🏽 👎🏽 ✊🏽 👊🏽 🤛🏽 🤜🏽 👏🏽 🙌🏽 👐🏽 🤲🏽 🙏🏽 ✍🏽 💅🏽 🤳🏽 💪🏽 🦵🏽 🦶🏽 👂🏽 🦻🏽 👃🏽 👶🏽 🧒🏽 👦🏽 👧🏽 🧑🏽 👱🏽 👨🏽 🧔🏽 🧔🏽‍♀️ 🧔🏽‍♂️ 👨🏽‍🦰 👨🏽‍🦱 👨🏽‍🦳 👨🏽‍🦲 👩🏽 👩🏽‍🦰 🧑🏽‍🦰 👩🏽‍🦱 🧑🏽‍🦱 👩🏽‍🦳 🧑🏽‍🦳 👩🏽‍🦲 🧑🏽‍🦲 👱🏽‍♀️ 👱🏽‍♂️ 🧓🏽 👴🏽 👵🏽 🙍🏽 🙍🏽‍♂️ 🙍🏽‍♀️ 🙎🏽 🙎🏽‍♂️ 🙎🏽‍♀️ 🙅🏽 🙅🏽‍♂️ 🙅🏽‍♀️ 🙆🏽 🙆🏽‍♂️ 🙆🏽‍♀️ 💁🏽 💁🏽‍♂️ 💁🏽‍♀️ 🙋🏽 🙋🏽‍♂️ 🙋🏽‍♀️ 🧏🏽 🧏🏽‍♂️ 🧏🏽‍♀️ 🙇🏽 🙇🏽‍♂️ 🙇🏽‍♀️ 🤦🏽 🤦🏽‍♂️ 🤦🏽‍♀️ 🤷🏽 🤷🏽‍♂️ 🤷🏽‍♀️ 🧑🏽‍⚕️ 👨🏽‍⚕️ 👩🏽‍⚕️ 🧑🏽‍🎓 👨🏽‍🎓 👩🏽‍🎓 🧑🏽‍🏫 👨🏽‍🏫 👩🏽‍🏫 🧑🏽‍⚖️ 👨🏽‍⚖️ 👩🏽‍⚖️ 🧑🏽‍🌾 👨🏽‍🌾 👩🏽‍🌾 🧑🏽‍🍳 👨🏽‍🍳 👩🏽‍🍳 🧑🏽‍🔧 👨🏽‍🔧 👩🏽‍🔧 🧑🏽‍🏭 👨🏽‍🏭 👩🏽‍🏭 🧑🏽‍💼 👨🏽‍💼 👩🏽‍💼 🧑🏽‍🔬 👨🏽‍🔬 👩🏽‍🔬 🧑🏽‍💻 👨🏽‍💻 👩🏽‍💻 🧑🏽‍🎤 👨🏽‍🎤 👩🏽‍🎤 🧑🏽‍🎨 👨🏽‍🎨 👩🏽‍🎨 🧑🏽‍✈️ 👨🏽‍✈️ 👩🏽‍✈️ 🧑🏽‍🚀 👨🏽‍🚀 👩🏽‍🚀 🧑🏽‍🚒 👨🏽‍🚒 👩🏽‍🚒 👮🏽 👮🏽‍♂️ 👮🏽‍♀️ 🕵🏽 🕵🏽‍♂️ 🕵🏽‍♀️ 💂🏽 💂🏽‍♂️ 💂🏽‍♀️ 🥷🏽 👷🏽 👷🏽‍♂️ 👷🏽‍♀️ 🤴🏽 👸🏽 👳🏽 👳🏽‍♂️ 👳🏽‍♀️ 👲🏽 🧕🏽 🤵🏽 🤵🏽‍♂️ 🤵🏽‍♀️ 👰🏽 👰🏽‍♂️ 👰🏽‍♀️ 🤰🏽 🤱🏽 👩🏽‍🍼 👨🏽‍🍼 🧑🏽‍🍼 👼🏽 🎅🏽 🤶🏽 🧑🏽‍🎄 🦸🏽 🦸🏽‍♂️ 🦸🏽‍♀️ 🦹🏽 🦹🏽‍♂️ 🦹🏽‍♀️ 🧙🏽 🧙🏽‍♂️ 🧙🏽‍♀️ 🧚🏽 🧚🏽‍♂️ 🧚🏽‍♀️ 🧛🏽 🧛🏽‍♂️ 🧛🏽‍♀️ 🧜🏽 🧜🏽‍♂️ 🧜🏽‍♀️ 🧝🏽 🧝🏽‍♂️ 🧝🏽‍♀️ 💆🏽 💆🏽‍♂️ 💆🏽‍♀️ 💇🏽 💇🏽‍♂️ 💇🏽‍♀️ 🚶🏽 🚶🏽‍♂️ 🚶🏽‍♀️ 🧍🏽 🧍🏽‍♂️ 🧍🏽‍♀️ 🧎🏽 🧎🏽‍♂️ 🧎🏽‍♀️ 🧑🏽‍🦯 👨🏽‍🦯 👩🏽‍🦯 🧑🏽‍🦼 👨🏽‍🦼 👩🏽‍🦼 🧑🏽‍🦽 👨🏽‍🦽 👩🏽‍🦽 🏃🏽 🏃🏽‍♂️ 🏃🏽‍♀️ 💃🏽 🕺🏽 🕴🏽 🧖🏽 🧖🏽‍♂️ 🧖🏽‍♀️ 🧗🏽 🧗🏽‍♂️ 🧗🏽‍♀️ 🏇🏽 🏂🏽 🏌🏽 🏌🏽‍♂️ 🏌🏽‍♀️ 🏄🏽 🏄🏽‍♂️ 🏄🏽‍♀️ 🚣🏽 🚣🏽‍♂️ 🚣🏽‍♀️ 🏊🏽 🏊🏽‍♂️ 🏊🏽‍♀️ ⛹🏽 ⛹🏽‍♂️ ⛹🏽‍♀️ 🏋🏽 🏋🏽‍♂️ 🏋🏽‍♀️ 🚴🏽 🚴🏽‍♂️ 🚴🏽‍♀️ 🚵🏽 🚵🏽‍♂️ 🚵🏽‍♀️ 🤸🏽 🤸🏽‍♂️ 🤸🏽‍♀️ 🤽🏽 🤽🏽‍♂️ 🤽🏽‍♀️ 🤾🏽 🤾🏽‍♂️ 🤾🏽‍♀️ 🤹🏽 🤹🏽‍♂️ 🤹🏽‍♀️ 🧘🏽 🧘🏽‍♂️ 🧘🏽‍♀️ 🛀🏽 🛌🏽 💑🏽 💏🏽 👫🏽 👭🏽 👬🏽 🫱🏽 🫲🏽 🫳🏽 🫴🏽 🫰🏽 🫵🏽 🫵🏽 🫶🏽 🤝🏽 🫅🏽 🫃🏽 🫄🏽",
    "dark brown skin" : "👋🏾 🤚🏾 🖐🏾 ✋🏾 🖖🏾 👌🏾 🤌🏾 🤏🏾 ✌🏾 🤞🏾 🤟🏾 🤘🏾 🤙🏾 👈🏾 👉🏾 👆🏾 🖕🏾 👇🏾 ☝🏾 👍🏾 👎🏾 ✊🏾 👊🏾 🤛🏾 🤜🏾 👏🏾 🙌🏾 👐🏾 🤲🏾 🙏🏾 ✍🏾 💅🏾 🤳🏾 💪🏾 🦵🏾 🦶🏾 👂🏾 🦻🏾 👃🏾 👶🏾 🧒🏾 👦🏾 👧🏾 🧑🏾 👱🏾 👨🏾 🧔🏾 🧔🏾‍♀️ 🧔🏾‍♂️ 👨🏾‍🦰 👨🏾‍🦱 👨🏾‍🦳 👨🏾‍🦲 👩🏾 👩🏾‍🦰 🧑🏾‍🦰 👩🏾‍🦱 🧑🏾‍🦱 👩🏾‍🦳 🧑🏾‍🦳 👩🏾‍🦲 🧑🏾‍🦲 👱🏾‍♀️ 👱🏾‍♂️ 🧓🏾 👴🏾 👵🏾 🙍🏾 🙍🏾‍♂️ 🙍🏾‍♀️ 🙎🏾 🙎🏾‍♂️ 🙎🏾‍♀️ 🙅🏾 🙅🏾‍♂️ 🙅🏾‍♀️ 🙆🏾 🙆🏾‍♂️ 🙆🏾‍♀️ 💁🏾 💁🏾‍♂️ 💁🏾‍♀️ 🙋🏾 🙋🏾‍♂️ 🙋🏾‍♀️ 🧏🏾 🧏🏾‍♂️ 🧏🏾‍♀️ 🙇🏾 🙇🏾‍♂️ 🙇🏾‍♀️ 🤦🏾 🤦🏾‍♂️ 🤦🏾‍♀️ 🤷🏾 🤷🏾‍♂️ 🤷🏾‍♀️ 🧑🏾‍⚕️ 👨🏾‍⚕️ 👩🏾‍⚕️ 🧑🏾‍🎓 👨🏾‍🎓 👩🏾‍🎓 🧑🏾‍🏫 👨🏾‍🏫 👩🏾‍🏫 🧑🏾‍⚖️ 👨🏾‍⚖️ 👩🏾‍⚖️ 🧑🏾‍🌾 👨🏾‍🌾 👩🏾‍🌾 🧑🏾‍🍳 👨🏾‍🍳 👩🏾‍🍳 🧑🏾‍🔧 👨🏾‍🔧 👩🏾‍🔧 🧑🏾‍🏭 👨🏾‍🏭 👩🏾‍🏭 🧑🏾‍💼 👨🏾‍💼 👩🏾‍💼 🧑🏾‍🔬 👨🏾‍🔬 👩🏾‍🔬 🧑🏾‍💻 👨🏾‍💻 👩🏾‍💻 🧑🏾‍🎤 👨🏾‍🎤 👩🏾‍🎤 🧑🏾‍🎨 👨🏾‍🎨 👩🏾‍🎨 🧑🏾‍✈️ 👨🏾‍✈️ 👩🏾‍✈️ 🧑🏾‍🚀 👨🏾‍🚀 👩🏾‍🚀 🧑🏾‍🚒 👨🏾‍🚒 👩🏾‍🚒 👮🏾 👮🏾‍♂️ 👮🏾‍♀️ 🕵🏾 🕵🏾‍♂️ 🕵🏾‍♀️ 💂🏾 💂🏾‍♂️ 💂🏾‍♀️ 🥷🏾 👷🏾 👷🏾‍♂️ 👷🏾‍♀️ 🤴🏾 👸🏾 👳🏾 👳🏾‍♂️ 👳🏾‍♀️ 👲🏾 🧕🏾 🤵🏾 🤵🏾‍♂️ 🤵🏾‍♀️ 👰🏾 👰🏾‍♂️ 👰🏾‍♀️ 🤰🏾 🤱🏾 👩🏾‍🍼 👨🏾‍🍼 🧑🏾‍🍼 👼🏾 🎅🏾 🤶🏾 🧑🏾‍🎄 🦸🏾 🦸🏾‍♂️ 🦸🏾‍♀️ 🦹🏾 🦹🏾‍♂️ 🦹🏾‍♀️ 🧙🏾 🧙🏾‍♂️ 🧙🏾‍♀️ 🧚🏾 🧚🏾‍♂️ 🧚🏾‍♀️ 🧛🏾 🧛🏾‍♂️ 🧛🏾‍♀️ 🧜🏾 🧜🏾‍♂️ 🧜🏾‍♀️ 🧝🏾 🧝🏾‍♂️ 🧝🏾‍♀️ 💆🏾 💆🏾‍♂️ 💆🏾‍♀️ 💇🏾 💇🏾‍♂️ 💇🏾‍♀️ 🚶🏾 🚶🏾‍♂️ 🚶🏾‍♀️ 🧍🏾 🧍🏾‍♂️ 🧍🏾‍♀️ 🧎🏾 🧎🏾‍♂️ 🧎🏾‍♀️ 🧑🏾‍🦯 👨🏾‍🦯 👩🏾‍🦯 🧑🏾‍🦼 👨🏾‍🦼 👩🏾‍🦼 🧑🏾‍🦽 👨🏾‍🦽 👩🏾‍🦽 🏃🏾 🏃🏾‍♂️ 🏃🏾‍♀️ 💃🏾 🕺🏾 🕴🏾 🧖🏾 🧖🏾‍♂️ 🧖🏾‍♀️ 🧗🏾 🧗🏾‍♂️ 🧗🏾‍♀️ 🏇🏾 🏂🏾 🏌🏾 🏌🏾‍♂️ 🏌🏾‍♀️ 🏄🏾 🏄🏾‍♂️ 🏄🏾‍♀️ 🚣🏾 🚣🏾‍♂️ 🚣🏾‍♀️ 🏊🏾 🏊🏾‍♂️ 🏊🏾‍♀️ ⛹🏾 ⛹🏾‍♂️ ⛹🏾‍♀️ 🏋🏾 🏋🏾‍♂️ 🏋🏾‍♀️ 🚴🏾 🚴🏾‍♂️ 🚴🏾‍♀️ 🚵🏾 🚵🏾‍♂️ 🚵🏾‍♀️ 🤸🏾 🤸🏾‍♂️ 🤸🏾‍♀️ 🤽🏾 🤽🏾‍♂️ 🤽🏾‍♀️ 🤾🏾 🤾🏾‍♂️ 🤾🏾‍♀️ 🤹🏾 🤹🏾‍♂️ 🤹🏾‍♀️ 🧘🏾 🧘🏾‍♂️ 🧘🏾‍♀️ 🛀🏾 🛌🏾 💑🏾 💏🏾 👫🏾 👭🏾 👬🏾 🫱🏾 🫲🏾 🫳🏾 🫴🏾 🫰🏾 🫵🏾 🫵🏾 🫶🏾 🤝🏾 🫅🏾 🫃🏾 🫄🏾",
    "black sin" : "👋🏿 🤚🏿 🖐🏿 ✋🏿 🖖🏿 👌🏿 🤌🏿 🤏🏿 ✌🏿 🤞🏿 🤟🏿 🤘🏿 🤙🏿 👈🏿 👉🏿 👆🏿 🖕🏿 👇🏿 ☝🏿 👍🏿 👎🏿 ✊🏿 👊🏿 🤛🏿 🤜🏿 👏🏿 🙌🏿 👐🏿 🤲🏿 🙏🏿 ✍🏿 💅🏿 🤳🏿 💪🏿 🦵🏿 🦶🏿 👂🏿 🦻🏿 👃🏿 👶🏿 🧒🏿 👦🏿 👧🏿 🧑🏿 👱🏿 👨🏿 🧔🏿 🧔🏿‍♀️ 🧔🏿‍♂️ 👨🏿‍🦰 👨🏿‍🦱 👨🏿‍🦳 👨🏿‍🦲 👩🏿 👩🏿‍🦰 🧑🏿‍🦰 👩🏿‍🦱 🧑🏿‍🦱 👩🏿‍🦳 🧑🏿‍🦳 👩🏿‍🦲 🧑🏿‍🦲 👱🏿‍♀️ 👱🏿‍♂️ 🧓🏿 👴🏿 👵🏿 🙍🏿 🙍🏿‍♂️ 🙍🏿‍♀️ 🙎🏿 🙎🏿‍♂️ 🙎🏿‍♀️ 🙅🏿 🙅🏿‍♂️ 🙅🏿‍♀️ 🙆🏿 🙆🏿‍♂️ 🙆🏿‍♀️ 💁🏿 💁🏿‍♂️ 💁🏿‍♀️ 🙋🏿 🙋🏿‍♂️ 🙋🏿‍♀️ 🧏🏿 🧏🏿‍♂️ 🧏🏿‍♀️ 🙇🏿 🙇🏿‍♂️ 🙇🏿‍♀️ 🤦🏿 🤦🏿‍♂️ 🤦🏿‍♀️ 🤷🏿 🤷🏿‍♂️ 🤷🏿‍♀️ 🧑🏿‍⚕️ 👨🏿‍⚕️ 👩🏿‍⚕️ 🧑🏿‍🎓 👨🏿‍🎓 👩🏿‍🎓 🧑🏿‍🏫 👨🏿‍🏫 👩🏿‍🏫 🧑🏿‍⚖️ 👨🏿‍⚖️ 👩🏿‍⚖️ 🧑🏿‍🌾 👨🏿‍🌾 👩🏿‍🌾 🧑🏿‍🍳 👨🏿‍🍳 👩🏿‍🍳 🧑🏿‍🔧 👨🏿‍🔧 👩🏿‍🔧 🧑🏿‍🏭 👨🏿‍🏭 👩🏿‍🏭 🧑🏿‍💼 👨🏿‍💼 👩🏿‍💼 🧑🏿‍🔬 👨🏿‍🔬 👩🏿‍🔬 🧑🏿‍💻 👨🏿‍💻 👩🏿‍💻 🧑🏿‍🎤 👨🏿‍🎤 👩🏿‍🎤 🧑🏿‍🎨 👨🏿‍🎨 👩🏿‍🎨 🧑🏿‍✈️ 👨🏿‍✈️ 👩🏿‍✈️ 🧑🏿‍🚀 👨🏿‍🚀 👩🏿‍🚀 🧑🏿‍🚒 👨🏿‍🚒 👩🏿‍🚒 👮🏿 👮🏿‍♂️ 👮🏿‍♀️ 🕵🏿 🕵🏿‍♂️ 🕵🏿‍♀️ 💂🏿 💂🏿‍♂️ 💂🏿‍♀️ 🥷🏿 👷🏿 👷🏿‍♂️ 👷🏿‍♀️ 🤴🏿 👸🏿 👳🏿 👳🏿‍♂️ 👳🏿‍♀️ 👲🏿 🧕🏿 🤵🏿 🤵🏿‍♂️ 🤵🏿‍♀️ 👰🏿 👰🏿‍♂️ 👰🏿‍♀️ 🤰🏿 🤱🏿 👩🏿‍🍼 👨🏿‍🍼 🧑🏿‍🍼 👼🏿 🎅🏿 🤶🏿 🧑🏿‍🎄 🦸🏿 🦸🏿‍♂️ 🦸🏿‍♀️ 🦹🏿 🦹🏿‍♂️ 🦹🏿‍♀️ 🧙🏿 🧙🏿‍♂️ 🧙🏿‍♀️ 🧚🏿 🧚🏿‍♂️ 🧚🏿‍♀️ 🧛🏿 🧛🏿‍♂️ 🧛🏿‍♀️ 🧜🏿 🧜🏿‍♂️ 🧜🏿‍♀️ 🧝🏿 🧝🏿‍♂️ 🧝🏿‍♀️ 💆🏿 💆🏿‍♂️ 💆🏿‍♀️ 💇🏿 💇🏿‍♂️ 💇🏿‍♀️ 🚶🏿 🚶🏿‍♂️ 🚶🏿‍♀️ 🧍🏿 🧍🏿‍♂️ 🧍🏿‍♀️ 🧎🏿 🧎🏿‍♂️ 🧎🏿‍♀️ 🧑🏿‍🦯 👨🏿‍🦯 👩🏿‍🦯 🧑🏿‍🦼 👨🏿‍🦼 👩🏿‍🦼 🧑🏿‍🦽 👨🏿‍🦽 👩🏿‍🦽 🏃🏿 🏃🏿‍♂️ 🏃🏿‍♀️ 💃🏿 🕺🏿 🕴🏿 🧖🏿 🧖🏿‍♂️ 🧖🏿‍♀️ 🧗🏿 🧗🏿‍♂️ 🧗🏿‍♀️ 🏇🏿 🏂🏿 🏌🏿 🏌🏿‍♂️ 🏌🏿‍♀️ 🏄🏿 🏄🏿‍♂️ 🏄🏿‍♀️ 🚣🏿 🚣🏿‍♂️ 🚣🏿‍♀️ 🏊🏿 🏊🏿‍♂️ 🏊🏿‍♀️ ⛹🏿 ⛹🏿‍♂️ ⛹🏿‍♀️ 🏋🏿 🏋🏿‍♂️ 🏋🏿‍♀️ 🚴🏿 🚴🏿‍♂️ 🚴🏿‍♀️ 🚵🏿 🚵🏿‍♂️ 🚵🏿‍♀️ 🤸🏿 🤸🏿‍♂️ 🤸🏿‍♀️ 🤽🏿 🤽🏿‍♂️ 🤽🏿‍♀️ 🤾🏿 🤾🏿‍♂️ 🤾🏿‍♀️ 🤹🏿 🤹🏿‍♂️ 🤹🏿‍♀️ 🧘🏿 🧘🏿‍♂️ 🧘🏿‍♀️ 🛀🏿 🛌🏿 💑🏿 💏🏿 👫🏿 👭🏿 👬🏿 🫱🏿 🫲🏿 🫳🏿 🫴🏿 🫰🏿 🫵🏿 🫵🏿 🫶🏿 🤝🏿 🫅🏿 🫃🏿 🫄🏿",
}

# Switch to pandas

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(extracted_data, columns=["uid", "datetime", "author", "message"])

In [ ]:
# test working with datetime
t = df.iloc[0].datetime
t.hour